# Naive Bayes Exercise

In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['figure.figsize'] = (12.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

# Fix the seed of the random number 
# generator so that your results will match ours
np.random.seed(1)

%load_ext autoreload
%autoreload 2

## Iris dataset

The iris dataset  has 4 features with continuous values. As such, we will be using a Gaussian distribution to describe our data. 

How many parameters do we need to estimate for NB classification (posterior)?


In [139]:
import pandas as pd

# loads csv file into a pandas dataframe
iris = pd.read_csv('iris.csv')

# pd.head() displays the first 5 elements
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [140]:
iris.groupby('species').describe()

petal_length  petal_width  sepal_length  sepal_width
species                                                               
setosa     count     50.000000    50.000000     50.000000    50.000000
           mean       1.464000     0.244000      5.006000     3.418000
           std        0.173511     0.107210      0.352490     0.381024
           min        1.000000     0.100000      4.300000     2.300000
           25%        1.400000     0.200000      4.800000     3.125000
           50%        1.500000     0.200000      5.000000     3.400000
           75%        1.575000     0.300000      5.200000     3.675000
           max        1.900000     0.600000      5.800000     4.400000
versicolor count     50.000000    50.000000     50.000000    50.000000
           mean       4.260000     1.326000      5.936000     2.770000
           std        0.469911     0.197753      0.516171     0.313798
           min        3.000000     1.000000      4.900000     2.000000
           25%        4.000000     1.200000      5.600000     2.525000
           50%        4.350000     1.300000      5.900000     2.800000
           75%        4.600000     1.500000      6.300000     3.000000
           max        5.100000     1.800000      7.000000     3.400000
virginica  count     50.000000    50.000000     50.000000    50.000000
           mean       5.552000     2.026000      6.588000     2.974000
           std        0.551895     0.274650      0.635880     0.322497
           min        4.500000     1.400000      4.900000     2.200000
           25%        5.100000     1.800000      6.225000     2.800000
           50%        5.550000     2.000000      6.500000     3.000000
           75%        5.875000     2.300000      6.900000     3.175000
           max        6.900000     2.500000      7.900000     3.800000

In [141]:
# convert species feature from type string to integer
iris["species"] = pd.Categorical(iris["species"]).codes
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


#### Data preparation

In [142]:
X = iris.values[:,:-1] # get everything before the species type as data X
y = iris.values[:,-1].astype(int) # get the last column (species) as label y (from float64 to int)

In [143]:
num_items = X.shape[0]
randIdx = np.arange(num_items)
# randomly shuffly the indices
np.random.shuffle(randIdx)

test_percentage_split = 0.5

num_test = np.ceil(num_items * test_percentage_split).astype(int)
X_test = X[randIdx[0:num_test]]
y_test = y[randIdx[0:num_test]]

X_train = X[randIdx[num_test:]]
y_train = y[randIdx[num_test:]]

## Naive Bayes Classifier (Gaussian)

Open naive_bayes.py and implement the TODOs for Gaussian distribution

In [144]:
from naive_bayes import NaiveBayes

In [149]:
gaussian_nb = NaiveBayes(distribution="gaussian")
gaussian_nb.train(X_train, y_train)

#### Calculate for the accuracy of the classifier

In [150]:
predictions = gaussian_nb.predict(X_test)
print(np.sum(y_test == predictions) / num_test)

(75, 3)
0.946666666667


## Spam/Not-Spam dataset

The spam/not-spam dataset are data taken from spam and non-spam emails. Based on the words used, we want to classify whether the email is spam or not spam.

In this dataset, we just look at how frequent a chosen 2,500 words appear per document.


How many parameters do we need to estimate for NB classification (posterior)?
2500



##### Data preparation

In [169]:
import os
train_spam_dir = "lingspam/spam-train/"
train_nonspam_dir = "lingspam/nonspam-train/"
test_spam_dir = "lingspam/spam-test/"
test_nonspam_dir = "lingspam/nonspam-test/"

global_word_dict = {}
train_doc_word_list = []
test_doc_word_list = []
for folder in [train_spam_dir, train_nonspam_dir, test_spam_dir, test_nonspam_dir]:
    for file in os.listdir(folder):
        tmp_doc_word = {}
        with open(folder + file,"r") as f:
            words = f.read().split(" ")

            for word in words:
                word = word.strip()
                if len(word) > 1:
                    if word in global_word_dict:
                        global_word_dict[word] += 1
                    else:
                        global_word_dict[word] = 1
                        
                    if word in tmp_doc_word:
                        tmp_doc_word[word] += 1
                    else:
                        tmp_doc_word[word] = 1
                
                
            if folder == train_spam_dir:
                train_doc_word_list.append({'words': tmp_doc_word, 'label': 1})
            elif folder == train_nonspam_dir:
                train_doc_word_list.append({'words': tmp_doc_word, 'label': 0})
            elif folder == test_spam_dir:
                test_doc_word_list.append({'words': tmp_doc_word, 'label': 1})
            elif folder == test_nonspam_dir:
                test_doc_word_list.append({'words': tmp_doc_word, 'label': 0})

In [170]:
sorted_words = []
sorted_counts = []
for key in sorted(global_word_dict, key=global_word_dict.get, reverse=True):
    sorted_words.append(key)
    sorted_counts.append(global_word_dict[key])
vocab_size = 2500
vocabulary = sorted_words[0:vocab_size]
vocab_counts = sorted_counts[0:vocab_size]

In [171]:
vocabulary[:10]

['email',
 'order',
 'address',
 'language',
 'report',
 'mail',
 'our',
 'university',
 'send',
 'program']

In [172]:
num_train = len(train_doc_word_list)
X_train = np.zeros((num_train, vocab_size))
y_train = np.zeros(num_train)
for i in range(num_train):  
    y_train[i] = train_doc_word_list[i]['label']
    
    for j in range(len(vocabulary)):
        word = vocabulary[j]
        if word in train_doc_word_list[i]['words']:
            X_train[i,j] = train_doc_word_list[i]['words'][word]
        else:
            X_train[i,j] = 0
            
num_test = len(test_doc_word_list)
X_test = np.zeros((num_test, vocab_size))
y_test = np.zeros(num_test)
for i in range(num_test):  
    y_test[i] = test_doc_word_list[i]['label']
    
    for j in range(len(vocabulary)):
        word = vocabulary[j]
        if word in test_doc_word_list[i]['words']:
            X_test[i,j] = test_doc_word_list[i]['words'][word]
        else:
            X_test[i,j] = 0

## Naive Bayes Classifier (Multinomial)

Open naive_bayes.py and implement the TODOs for multinomial distribution

In [189]:
multinomial_nb = NaiveBayes(distribution="multinomial")
multinomial_nb.train(X_train, y_train)

In [190]:
pred = multinomial_nb.predict(X_test)

[0.5007153075822603, 0.49928469241773965]
(260,)


In [191]:
print(np.sum(y_test == pred) / num_test)

0.980769230769
